In [3]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [4]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip
!rm *.zip

--2024-04-13 09:14:03--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  3.68MB/s    in 7.0s    

2024-04-13 09:14:11 (3.57 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [5]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'
# import csv data into dataframes
df_books = pd.read_csv(
	books_filename,
	encoding = "ISO-8859-1",
	sep=";",
	header=0,
	names=['isbn', 'title', 'author'],
	usecols=['isbn', 'title', 'author'],
	dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
	ratings_filename,
	encoding = "ISO-8859-1",
	sep=";",
	header=0,
	names=['user', 'isbn', 'rating'],
	usecols=['user', 'isbn', 'rating'],
	dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [6]:
#Merging ratings with books to simplify work
df = pd.merge(df_ratings, df_books, on='isbn')
df = df.drop(columns=['author'])#not useing authors so lets remvoe them
df.drop_duplicates(['title','user'])

#remove users w/less than 200 ratings & books w/less than 100 ratings
user_count = df_ratings['user'].value_counts()
book_count = df_ratings['isbn'].value_counts()
user_mask = user_count[user_count >= 200].index
book_mask = book_count[book_count >= 100].index
valid_mask = (df['user'].isin(user_mask)) & (df['isbn'].isin(book_mask))
df = df[valid_mask]

In [7]:
#create matrix for training
table = df.pivot_table(index='title', columns='user', values='rating').fillna(0)
matrix = csr_matrix(table.values)
#metric needs to be cosine as the expected values in the test are less than 1
model = NearestNeighbors(n_neighbors=5, metric='cosine')
model.fit(matrix)

NearestNeighbors(metric='cosine')

In [8]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
	index = table.index.get_loc(book)
	distances, indices = model.kneighbors(table.iloc[index,:].values.reshape(1,-1))
	distances = distances.flatten()
	indices = indices.flatten()
	
	recommended_books = []#[book,distance]
	#rest wants the list reversed, so we'll go through the neighbors backwards
	for i in range(len(distances)-1, 1,-1):
		recommended_books.append([table.index[indices[i]], distances[i]])
	recommended_books = [book,recommended_books]
	return recommended_books

In [9]:
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566],
  ['Interview with the Vampire', 0.73450685],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384]]]

In [10]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
	test_pass = True
	recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
	if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
		test_pass = False
	recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
	recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
	for i in range(2): 
		if recommends[1][i][0] not in recommended_books:
			test_pass = False
		if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
			test_pass = False
	if test_pass:
		print("You passed the challenge! 🎉🎉🎉🎉🎉")
	else:
		print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075]]]
You passed the challenge! 🎉🎉🎉🎉🎉
